# MD trajectory versus inherent structure trajectory site assignments

This notebook compares the a site-occupation analysis of the Li<sub>6</sub>PS<sub>5</sub><i>X</i> molecular dynamics simulation trajectories from the "raw" molecular dynamics trajectories ("actual" trajectories) and the set of quenched structures ("inherent" trajectories).

In [1]:
import yaml
from pymatgen.io.vasp import Xdatcar
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from site_analysis_functions import tetrahedral_site_analysis
from site_analysis_functions import site_populations
import numpy as np

import sys
sys.path.insert(0, "../../scripts/")
from utils import flatten_list

In [2]:
with open('../md_runs.yaml', 'r') as f:
    md_runs = yaml.safe_load(f)
print(md_runs)

{'Li6PS5I': {'0p': [1, 2, 3, 4, 5, 6], '50p': [1, 2, 3, 4, 5, 6, 7], '100p': [1, 2, 3, 4, 5, 6, 7]}, 'Li6PS5Cl': {'0p': [1, 2, 3, 4, 5], '50p': [1, 2, 3, 4, 5, 6], '100p': [1, 2, 3, 4, 5, 6]}}


In [ ]:
data_dir = '../../data'

x_spec = {'Li6PS5I': 'I', 'Li6PS5Cl': 'Cl'}
disorder_selection = {'Li6PS5I': '0p', 'Li6PS5Cl': '50p'}

inherent_trajectory = {}
inherent_populations = {}
actual_trajectory = {}
actual_populations = {}
for system in md_runs:
    inherent_trajectory[system] = {}
    inherent_populations[system] = {}
    actual_trajectory[system] = {}
    actual_populations[system] = {}
    disorder = disorder_selection[system]
    runs = md_runs[system][disorder]
    print(f'Analysing {system}: {disorder}')
    xdatcar_filenames = [f'{data_dir}/{system}/{disorder}/run{i}/inherent_XDATCAR.gz' for i in runs]
    xdatcars = (Xdatcar(f) for f in xdatcar_filenames)
    structures = flatten_list([x.structures for x in xdatcars])
    inherent_trajectory[system][disorder] = tetrahedral_site_analysis(
        structures, x_spec[system])
    inherent_populations[system][disorder] = site_populations(
        inherent_trajectory[system][disorder])

    xdatcar_filenames = [f'{data_dir}/{system}/{disorder}/run{i}/actual_XDATCAR.gz' for i in runs]
    xdatcars = (Xdatcar(f) for f in xdatcar_filenames)
    structures = flatten_list([x.structures for x in xdatcars])
    actual_trajectory[system][disorder] = tetrahedral_site_analysis(
        structures, x_spec[system])
    actual_populations[system][disorder] = site_populations(
        actual_trajectory[system][disorder])

Analysing Li6PS5I: 0p
max mapping disp = 0.26 Angstroms


100%|██████████| 700/700 [14:22<00:00,  1.23s/ steps]


max mapping disp = 0.77 Angstroms


 17%|█▋        | 122/700 [03:18<14:46,  1.53s/ steps]

In [ ]:
from matplotlib import rcParams

rcParams['font.serif'] = "Iowan Old Style"
rcParams['font.family'] = "serif"

rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.rm'] = 'Iowan Old Style'
rcParams['mathtext.it'] = 'Iowan Old Style:italic'
rcParams['mathtext.bf'] = 'Iowan Old Style:bold'


rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams.update({'font.size': 16})
rcParams['text.usetex']=False

In [ ]:
def site_mapping_stuff(it_at, at_at):
    # first, how many sites are we dealing with?
    site_ids, ind = np.unique(np.concatenate((at_at, it_at)), return_index=True)
    site_ids = site_ids[np.argsort(ind)]
    site_mapping = {s: i for i, s in enumerate(site_ids)}
    mapped_it_at = [site_mapping[s] for s in it_at]
    mapped_at_at = [site_mapping[s] for s in at_at]
    return mapped_it_at, mapped_at_at

In [ ]:
def plot_sites(it_at, at_at, ax):
    a, b = site_mapping_stuff(it_at, at_at)
    ax.plot(timesteps, b, '-', linewidth=2, label='raw coordinates', alpha=0.8)
    ax.plot(timesteps, a, '-', linewidth=2, label='inherent structures', alpha=1.0)
    ax.legend()
    ind = list(set(a) | set(b))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_yticks(np.arange(0, max(ind)+1, (max(a)+4)//5))
    ax.set_xlim([0,70])
    ax.set_xlabel('Time / ps')
    ax.set_ylabel('Site index')
    return

In [ ]:
it = inherent_trajectory['Li6PS5I']['0p']
at = actual_trajectory['Li6PS5I']['0p']

it_at = np.array(it.at)
at_at = np.array(at.at)

timesteps = np.arange(1, len(it)+1)/10

for i in range(192):
    print(f'i(Li) = {i}')
    fig, ax = plt.subplots(1,1,figsize=(10,3))
    plot_sites(it_at[:,i], at_at[:,i], ax)
    plt.show()

    print('–––––––––––––––––––––––––––––––––––––––––\n')

In [ ]:
it = inherent_trajectory['Li6PS5Cl']['50p']
at = actual_trajectory['Li6PS5Cl']['50p']

it_at = np.array(it.at)
at_at = np.array(at.at)

timesteps = np.arange(1, len(it)+1)/10

for i in range(192):
    print(f'i(Li) = {i}')
    fig, ax = plt.subplots(1,1,figsize=(10,3))
    plot_sites(it_at[:,i], at_at[:,i], ax)
    plt.show()

    print('–––––––––––––––––––––––––––––––––––––––––\n')

In [ ]:
# Figure for SI
fig, ax = plt.subplots(2, 1, figsize=(10,7))

it = inherent_trajectory['Li6PS5I']['0p']
at = actual_trajectory['Li6PS5I']['0p']
it_at = np.array(it.at)
at_at = np.array(at.at)
i = 2
plot_sites(it_at[:,i], at_at[:,i], ax[0])
ax[0].legend(loc='upper left')
ax[0].set_title('Li$_6$PS$_5$I 0%', loc='left', pad=15, fontsize=18)

it = inherent_trajectory['Li6PS5Cl']['50p']
at = actual_trajectory['Li6PS5Cl']['50p']
it_at = np.array(it.at)
at_at = np.array(at.at)
i = 87
plot_sites(it_at[:,i], at_at[:,i], ax[1])
ax[1].legend(loc='upper left')
ax[1].set_title('Li$_6$PS$_5$Cl 50%', loc='left', pad=15, fontsize=18)

plt.tight_layout()

plt.savefig('../../figures/inherent_vs_actual_sites_comparison.pdf')

plt.show()

In [ ]:
%load_ext version_information

%version_information yaml, pymatgen, tqdm, numpy, site_analysis, matplotlib